In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
import pandas as pd
from pathlib import Path
import src.config
from deltalake import DeltaTable

In [ ]:
wh_allroles = DeltaTable(src.config.BRONZE / "wh_allroles").to_pandas()


In [ ]:
ALERTS_CURRENT_FILE = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Credit Loan Review\Alerts\Production\Output\alerts.xlsx")
alerts = pd.read_excel(ALERTS_CURRENT_FILE)

In [ ]:
alerts

In [ ]:
guar = wh_allroles[wh_allroles['acctrolecd'] == 'GUAR'].copy()

In [ ]:
guar = guar[~(guar['persnbr'].isnull())].copy()

In [ ]:
guar

In [ ]:
guar['persnbr'] = guar['persnbr'].astype('Int64').astype(str)


In [ ]:
merged_eligible = alerts.merge(guar, on='acctnbr', how='left')

In [ ]:
merged_eligible

In [ ]:
PERMISSION_PATH = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Credit Loan Review\Alerts\Production\assets\xactus\pfs_permission.csv")
permission = pd.read_csv(PERMISSION_PATH)

In [ ]:
permission

In [ ]:
permission['persnbr'] = permission['persnbr'].astype(str)

In [ ]:
check = merged_eligible.merge(permission, how='outer', on='persnbr', indicator=True)

In [ ]:
check

In [ ]:
permission = check[check['_merge'] != 'left_only'].copy()
permission = permission[[
    'acctnbr',
    'ownername',
    'product',
    'loanofficer',
    'inactivedate',
    'persnbr'
]].copy()

In [ ]:
# Get bronze permission tables to get names
wh_pers = DeltaTable(src.config.BRONZE / "wh_pers").to_pandas()
assert wh_pers['persnbr'].is_unique, "Persnbr not unique in wh_pers"

In [ ]:
wh_pers

In [ ]:
wh_pers = wh_pers[['persnbr','perssortname','age']].copy()

In [ ]:
wh_pers['persnbr'] = wh_pers['persnbr'].astype(str)
permission_detail = permission.merge(wh_pers, on='persnbr', how='left')

In [ ]:
permission_detail = permission_detail[~(permission_detail['persnbr'].isnull())].copy()

In [ ]:
permission_detail

In [ ]:
permission_detail['acctnbr'] = permission_detail['acctnbr'].astype('Int64').astype(str)

In [ ]:
permission_detail

In [ ]:
people = permission_detail[[
    'persnbr'
]].copy()



In [ ]:
import cdutils.deduplication

dedupe_list = [
    {'df': people, 'field': 'persnbr'}
]

people = cdutils.deduplication.dedupe(dedupe_list)

In [ ]:
assert people['persnbr'].is_unique, "Dupes"

In [ ]:

import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore

def fetch_data():
    pers = text("""
    SELECT 
        a.PERSNBR,
        a.FIRSTNAME,
        a.LASTNAME,
        a.DATEBIRTH,
        a.DEATHNOTIFICATIONDATE
    FROM 
        OSIBANK.PERS a
    """)
    
    viewperstaxid = text("""
    SELECT 
        *
    FROM 
        OSIBANK.VIEWPERSTAXID
    """)
    queries = [
        {'key':'pers', 'sql':pers, 'engine':1},
        {'key':'viewperstaxid', 'sql':viewperstaxid, 'engine':1}
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [ ]:
data = fetch_data()

In [ ]:
pers = data['pers'].copy()

In [ ]:
pers.info()

In [ ]:
pers

In [ ]:
assert pers['persnbr'].is_unique, "Dupes"

In [ ]:
pers = pers[[
    'persnbr',
    'firstname',
    'lastname'
]].copy()

In [ ]:
pers

In [ ]:
pers['persnbr'] = pers['persnbr'].astype(str)

In [ ]:
people = people.merge(pers, how='left', on='persnbr')

In [ ]:
people

In [ ]:
viewperstaxid = data['viewperstaxid'].copy()

In [ ]:
viewperstaxid.info()

In [ ]:
assert viewperstaxid['persnbr'].is_unique, "Dupes"

In [ ]:
viewperstaxid['persnbr'] = viewperstaxid['persnbr'].astype(str)

In [ ]:
viewperstaxid

In [ ]:
people = people.merge(viewperstaxid, how='left', on='persnbr')

In [ ]:
people

In [ ]:
people.info()

In [ ]:
persaddruse = DeltaTable(src.config.BRONZE / "persaddruse").to_pandas()

In [ ]:
persaddruse

In [ ]:
persaddruse = persaddruse[persaddruse['addrusecd'] == "PRI"].copy()

In [ ]:
persaddruse = persaddruse[['persnbr','addrnbr']].copy()

In [ ]:
persaddruse.info()

In [ ]:
persaddruse['persnbr'] = persaddruse['persnbr'].astype(str)
persaddruse['addrnbr'] = persaddruse['addrnbr'].astype(str)

In [ ]:
address = DeltaTable(src.config.SILVER / "address").to_pandas()

In [ ]:
address.info()

In [ ]:
address

In [ ]:
persaddruse = persaddruse.merge(address, on='addrnbr', how='left')

In [ ]:
persaddruse

In [ ]:
assert persaddruse['persnbr'].is_unique, "Fail"

In [ ]:
persaddruse = persaddruse[['persnbr','Full_Street_Address','cityname','statecd','zipcd']].copy()

In [ ]:
people = people.merge(persaddruse, how='left', on='persnbr')

In [ ]:
people

In [ ]:
OUTPUT_PATH = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Credit Loan Review\Alerts\Production\Output\xactus_ready.xlsx")
people.to_excel(OUTPUT_PATH, index=False)